In [86]:
#DL Model
import os, fnmatch
import sys
import librosa
import keras
import numpy as np
from keras.models import load_model
from sklearn import metrics

#ML Model
import pandas as pd 

from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn.ensemble import VotingClassifier
import scikitplot as skplt
from sklearn.metrics import confusion_matrix,roc_curve,roc_auc_score, accuracy_score, recall_score, f1_score, precision_score
import matplotlib.pyplot as plt
%matplotlib inline


# import cleaning
from sklearn.feature_selection import SelectKBest, f_classif
import pickle
import datetime

# Pre-processing

## Helping Functions

In [2]:
def extract_features(audio_path, offset):
	y, sr = librosa.load(audio_path, offset=offset, duration=3)
	S = librosa.feature.melspectrogram(
	y, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
	mfccs = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=40)
	# mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
	return mfccs

def most_freq_dl(list):
    # print("Ok1")
    return np.bincount(list).argmax()
            
MAX_SOUND_CLIP_DURATION=12 #sec  
def audio_norm(data):
    max_data = np.max(data)
    min_data = np.min(data)
    data = (data-min_data)/(max_data-min_data+0.0001)
    return data-0.5

# get audio data without padding highest qualify audio
def load_file_data_without_change(folder,file_names, duration=3, sr=16000):
    input_length=sr*duration
    # function to load files and extract features
    # file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("load file ",sound_file)
            # use kaiser_fast technique for faster extraction
            X, sr = librosa.load( sound_file,res_type='kaiser_fast') 
            dur = librosa.get_duration(y=X, sr=sr)
            # extract normalized mfcc feature from data
            mfccs_ml = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T,axis=0) 
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)
        feature = np.array(mfccs_ml).reshape([-1,1])
        data.append(feature)
    return data


# get audio data with a fix padding may also chop off some file
def load_file_data (folder,file_names, duration=12, sr=16000):
    input_length=sr*duration
    # function to load files and extract features
    # file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("load file ",sound_file)
            # use kaiser_fast technique for faster extraction
            X, sr = librosa.load( sound_file, sr=sr, duration=duration,res_type='kaiser_fast') 
            dur = librosa.get_duration(y=X, sr=sr)
            # pad audio file same duration
            if (round(dur) < duration):
                print ("fixing audio lenght :", file_name)
                y = librosa.util.fix_length(X, input_length)                
            #normalized raw audio 
            # y = audio_norm(y)            
            # extract normalized mfcc feature from data
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T,axis=0)             
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)        
        feature = np.array(mfccs).reshape([-1,1])
        data.append(feature)
    return data

## file to test

In [63]:
# file to test
ml_upload_folder_path = 'C:/Users/Manisha/BE_Project/Physionet_Dataset/training/training-a/'
classify_file = 'C:/Users/Manisha/BE_Project/Physionet_Dataset/training/training-a/a0011.wav'

## DL 

In [64]:
x_test_dl = []
x_test_dl.append(extract_features(classify_file, 0.5))
x_test_dl = np.asarray(x_test_dl)
# print(x_test_dl.shape[0])
# print(x_test_dl.shape[1])
# print(x_test_dl.shape[2])
# print(x_test_dl.shape[3])
x_test_dl = x_test_dl.reshape(x_test_dl.shape[0], x_test_dl.shape[1], x_test_dl.shape[2], 1)
model = keras.models.load_model('heartbeat_classifier_categorical_crossentropy.h5')
pred_class = model.predict_classes(x_test_dl, verbose=1)
CNNprob = model.predict_proba(x_test_dl, verbose=1)
# CNNprob = CNNprob[:,1:]
# print(CNNprob)
CNNpred = np.argmax(CNNprob, axis=1)
CNNpred = CNNpred + 1
# print(CNNpred)
print(type(pred_class))
scores =[]
scores = [accuracy_score(CNNpred,pred_class)]

# print(accuracy_score(CNNpred,[0]))

1/1 [==============================] - 0s 11ms/step
<class 'numpy.ndarray'>


In [65]:
test_sounds = load_file_data(folder=ml_upload_folder_path,file_names=['a0011.wav'], duration=MAX_SOUND_CLIP_DURATION)
test_labels = [-1 for items in test_sounds]
testing_data_rf = np.squeeze(test_sounds)
# print(testing_data_rf)
testing_data_rf = testing_data_rf.reshape(1, -1)
# loaded_model = pickle.load(open('ml_classifier_random_forest.pkl', 'rb'))
# prediction_rf = loaded_model.predict(testing_data_rf)
# KNNProb = loaded_model.predict_proba(testing_data_rf)
# print(type(prediction_rf))

load file  C:/Users/Manisha/BE_Project/Physionet_Dataset/training/training-a/a0011.wav


In [46]:
# def test_el(x_test_dl, testing_data_rf, dl_prediction_labels):
    
# #     KNNmodel = MLmodel
# #     MLPmodel = DL1model
# #     CNNmodel = DL2model
    
#     #Load saved models and PCA transformations
#     MLmodel = pickle.load(open('ml_classifier_random_forest.pkl', 'rb'))
#     DL1model = keras.models.load_model('heartbeat_classifier_binary_crossentropy.h5')
#     DL2model = keras.models.load_model('heartbeat_classifier_categorical_crossentropy.h5')
    
#     #Make predictions with test data
# #     scores = []
   
#     DL1pred = DL1model.predict_classes(x_test_dl)
#     DL1prob = DL1model.predict_proba(x_test_dl)
# #     scores += [accuracy_score(DL1pred,dl_prediction_labels)]
#     DL2prob = DL2model.predict(x_test_dl)
#     DL2prob = DL2prob[:,1:]
#     DL2pred = np.argmax(DL2prob, axis=1)
#     DL2pred = DL2pred + 1
# #     scores += [accuracy_score(DL2pred,dl_prediction_labels)]
    
#     MLpred = MLmodel.predict(testing_data_rf)
#     MLprob = MLmodel.predict_proba(testing_data_rf)
# #     scores += [accuracy_score(MLpred,dl_prediction_labels)]
    
#     #Ensemble learning/voting system
#     final_pred = []
    
#     for i in range(len(MLpred)):
#         print("DL2pred: ", DL2pred[i], "DL1pred: ", DL1pred[i], "MLpred: ", MLpred[i])
#         if MLpred[i]==2:
#             MLpred[i] = 0
#         if DL2pred[i]==MLpred[i] and MLpred[i]==DL1pred[i]:
#             final_pred += [DL2pred[i]]
#         elif DL2pred[i]==MLpred[i] or MLpred[i]==DL1pred[i] or DL2pred[i]==DL1pred[i]:
#             if DL2pred[i]==MLpred[i] or DL2pred[i]==DL1pred[i]:
#                 final_pred += [DL2pred[i]]
#             elif MLpred[i]==DL2pred[i] or MLpred[i]==DL1pred[i]:
#                 final_pred += [MLpred[i]]
#             elif DL1pred[i]==DL2pred[i] or DL1pred[i]==MLpred[i]:
#                 final_pred += [DL1pred[i]]
#             else:
#                 print('bug')
#         else:
#             print("DL2prob[i]: ", DL2prob[i], "DL1prob[i]:", DL1prob[i], "MLprob[i]: ", MLprob[i])
#             DL1prob_max = DL1prob.max()
#             MLprob_max = MLprob.max()
#             prob = DL2prob[i] + DL1prob_max + MLprob_max
#             pred = np.argmax(prob)
#             pred = pred+1
#             final_pred += [pred]
    
#     #Outputs
#     final_pred = np.array(final_pred)
# #     score = accuracy_score(final_pred,dl_prediction_labels)
# #     cm = confusion_matrix(final_pred,dl_prediction_labels)
#     return final_pred

In [47]:
from collections import Counter
  
def most_frequent(List):
    occurence_count = Counter(List)
#     print("occurence_count: ",occurence_count)
    return occurence_count.most_common(1)[0][0], occurence_count

In [48]:
my_list = [0,0,0,0,1, 1, 1, 1]
most_freq_item, occ_cnt = most_frequent(my_list)
print(occ_cnt[0] == occ_cnt[1])
print(occ_cnt[0])

occurence_count:  Counter({0: 4, 1: 4})
True
4


In [98]:
def test_el(x_test_dl, testing_data_rf, dl_prediction_labels):
    
#     KNNmodel = ML1model
#     MLPmodel = DL1model
#     CNNmodel = DL2model
    
    #Load saved models and PCA transformations
    ML1model = pickle.load(open('ml_classifier_random_forest.pkl', 'rb'))
    ML2model = pickle.load(open('ml_classifier_LogisticRegression.pkl', 'rb'))
    DL1model = keras.models.load_model('heartbeat_classifier_binary_crossentropy.h5')
    DL2model = keras.models.load_model('heartbeat_classifier_categorical_crossentropy.h5')
    
    #Make predictions with test data
    scores = []
   
    DL1pred = DL1model.predict_classes(x_test_dl)
    DL1prob = DL1model.predict_proba(x_test_dl)

    DL2prob = DL2model.predict(x_test_dl)
    DL2prob = DL2prob[:,1:]
    DL2pred = np.argmax(DL2prob, axis=1)
    DL2pred = DL2pred + 1

    
    MLpred = ML1model.predict(testing_data_rf)
    MLprob = ML1model.predict_proba(testing_data_rf)
    
    ML2pred = ML2model.predict(testing_data_rf)
    ML2prob = ML2model.predict_proba(testing_data_rf)
    
    scores += [accuracy_score(MLpred,dl_prediction_labels)]
    scores += [accuracy_score(ML2pred,dl_prediction_labels)]
    scores += [accuracy_score(DL1pred,dl_prediction_labels)]
    scores += [accuracy_score(DL2pred,dl_prediction_labels)]

    print("Scores: ", scores)
    
    
    #Ensemble learning/voting system
    final_pred = []
    
    
    for i in range(len(MLpred)):
#         print("DL2pred: ", DL2pred[i], "DL1pred: ", DL1pred[i], "MLpred: ", MLpred[i])
        if MLpred[i]==2:
            MLpred[i] = 0
            
        if ML2pred[i]==2:
            ML2pred[i] = 0
            
        final_pred.append(MLpred[i])
        final_pred.append(ML2pred[i])
        final_pred.append(DL1pred[i])
        final_pred.append(DL2pred[i])

        print("final_pred:", final_pred)
        to_pass = final_pred
        
        most_freq_item, occ_cnt = most_frequent(final_pred)

        if occ_cnt[0]==occ_cnt[1]:
            # soft voting
            DL1prob_max = DL1prob.max()
            MLprob_max = MLprob.max()
            ML2prob_max = ML2prob.max()
        
            prob_all = []
            prob_all.append(MLprob_max)
            prob_all.append(ML2prob_max)
            prob_all.append(DL1prob_max)
            prob_all.append(DL2prob[i])
            
            prob_all_max = np.argmax(prob_all)
#             print("prob_all : ",prob_all)
#             print("Prob_all_max:", prob_all_max)
            return_ele =final_pred[prob_all_max]
            
#             print("return_ele: ",return_ele)
            final_pred =[]
            final_pred.append(return_ele)
            
        else:
            # hard voting
            final_pred = []
            final_pred.append(most_freq_item)
    
    #Outputs
    final_pred = np.array(final_pred)
    dl_prediction_labels = np.array(dl_prediction_labels)
    
#     score = accuracy_score(final_pred,dl_prediction_labels)
#     cm = confusion_matrix(final_pred,dl_prediction_labels)
#     print("confusion_matrix")
#     print(cm)
    
    
#     fpr_log, tpr_log, thresholds_log = roc_curve(dl_prediction_labels, MLpred)
#     fpr_knn, tpr_knn, thresholds_knn = roc_curve(dl_prediction_labels, ML2pred)
#     fpr_tree, tpr_tree, thresholds_tree = roc_curve(dl_prediction_labels, DL1pred)
#     fpr_rfc, tpr_rfc, thresholds_rfc = roc_curve(dl_prediction_labels, DL1pred)
    
    
    
    
    
#     fpr_ens, tpr_ens, thresholds_ens = roc_curve(dl_prediction_labels, final_pred)
#     plt.plot(fpr_log, tpr_log,label = 'ML1')
#     plt.plot(fpr_knn, tpr_knn,label = 'ML2')
#     plt.plot(fpr_tree, tpr_tree,label = 'DL1')
#     plt.plot(fpr_rfc, tpr_rfc,label = 'DL2')
    
#     plt.plot(fpr_ens, tpr_ens,label = 'Ensemble')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.legend()
#     plt.title('ROC Curve no GridSearch')
#     plt.show()
#     print("AUC Ensemble: {}".format(roc_auc_score(dl_prediction_labels, final_pred)))
    return final_pred, score

In [99]:
dl_prediction_labels = pred_class

final_pred, score = test_el(x_test_dl, testing_data_rf, dl_prediction_labels)

print("final output: ", final_pred)
print("final score: ", score)
# print("confusion_matrix:", cm)


Scores:  [0.0, 0.0, 0.0, 1.0]
final_pred: [0, 0, 0, 1]
occurence_count:  Counter({0: 3, 1: 1})
final output:  [0]
final score:  0.0
